In [1]:
import torch

In [2]:
# Download data
# https://files.pushshift.io/hackernews/ (first file)

In [43]:
# Convert to HDF5
import bz2, json
from json import JSONDecodeError
b = bz2.BZ2File('data/HNI_2006-10.bz2')
bs = b.read()
bs[:100]

b'{"by":"pg","descendants":15,"id":1,"kids":[487171,15,234509,454410,82729],"retrieved_on":1525541947,'

In [44]:
bs.split(b'\n')[2]

b'{"by":"phyllis","descendants":0,"id":3,"kids":[454412,531602],"retrieved_on":1525541948,"score":7,"time":1160419233,"title":"Woz Interview: the early days of Apple","type":"story","url":"http:\\/\\/www.foundersatwork.com\\/stevewozniak.html"}'

In [48]:
bj = []
for l in bs.split(b'\n'):
    try:
        bj.append(json.loads(l))
    except JSONDecodeError:
        pass
[x for x in bj if x['type'] == 'comment'][:2]

[{'by': 'sama',
  'id': 15,
  'kids': [17, 454424],
  'parent': 1,
  'retrieved_on': 1525541949,
  'text': '&#34;the rising star of venture capital&#34; -unknown VC eating lunch on SHR',
  'time': 1160423461,
  'type': 'comment'},
 {'by': 'pg',
  'id': 17,
  'kids': [1079, 454426],
  'parent': 15,
  'retrieved_on': 1525541950,
  'text': 'Is there anywhere to eat on Sandhill Road?',
  'time': 1160423565,
  'type': 'comment'}]

In [ ]:
# Tag
# Maybe things like links should be tagged

In [69]:
# Tokenize
text_list = []
user_list = []
for (text, user) in ((x['text'], x['by']) for x in bj if x['type'] == 'comment'):
    text_list.append(text.split())
    user_list.append(user)


In [5]:
# W2V
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
# path = get_tmpfile("word2vec.model")
model = Word2Vec(text_list, size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [82]:
# Index
tok_list = []
for text in text_list:
    tok_list.append([])
    for tok in text:
        try:
            tok_list[-1].append(1+model.wv.index2word.index(tok))
        except:
            tok_list[-1].append(0)
            
    tok_list[-1] = tok_list[-1] + [0]*10 
    tok_list[-1] = tok_list[-1][:10] 
            
            
tok_list[5]

[88, 13, 0, 0, 0, 0, 0, 0, 0, 0]

In [84]:
# Define Model
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
n_letters = len(model.wv.index2word)
n_categories = 2
rnn = RNN(n_letters, n_hidden, n_categories)
# rnn.cuda()

In [85]:
# Define Model
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(CNN, self).__init__()

#         self.i2h = nn.Conv1d(input_size, )
        self.i2o = nn.Linear(input_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
#         output = self.i2h(combined)
        output = self.i2o(input)
        output = self.softmax(output)
        return output

input_size = len(model.wv.index2word)
n_categories = 2
cnn = CNN(input_size, n_categories)

In [ ]:
input = letterToTensor('A')

output, next_hidden = rnn(input, hidden)

In [8]:
# Train Model


In [9]:
# Test Model

In [10]:
# New Data